In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Setup Logging
import logging
logging_format = '%(asctime)s - %(levelname)s - %(message)s'
logging.basicConfig(level=logging.INFO,filename=None, format=logging_format)
log = logging.getLogger(__name__)
    
from minimal_bert import preproc as pp
from minimal_bert import modelling as mod
from minimal_bert import utils
from minimal_bert.utils import prepare_and_save_bert_encoded_df as prep_bert
from minimal_bert.utils import load_bert_and_prep_for_training as load_bert

# Sklearny
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [3]:
# Bulk Process All Articles
# prep_bert(maxlen=300)

# Split Test Set From All Articles
X_train, X_test, y_train, y_test = load_bert(None,0.1)
log.info(f"Testing Shape = {y_train.shape}")

2019-06-10 13:48:53,252 - INFO - Testing Shape = (31397, 24)


In [ ]:
# Create Parameter Grid
n_sample_li = [1000, 5000, 10000, 20000, 30000, None]
n_hidden = [32, 64, 128, 256]
param_grid = {'n_hidden': n_hidden, 'number_of_texts': n_sample_li}
param_grid = list(ParameterGrid(param_grid))

n_samples_li = []
n_hidden_li = []
val_loss_li = []
val_acc_li = []
val_prec_li = []
val_rec_li = []
test_loss_li = []
test_acc_li = []
test_prec_li = []
test_rec_li = []
for ix, grid in enumerate(param_grid):
    
    # Get Data for sample size
    X = X_train[0:grid['number_of_texts']].copy()
    Y = y_train[0:grid['number_of_texts']].copy()
    Xtr, Xval, ytr, yval = train_test_split(
        X, Y, test_size=0.25, shuffle=True, random_state=42)
    
    # Train Model
    model, history = mod.train_and_return_model(
        Xtr, Xval, ytr, yval, grid['n_hidden'])
    
    # Extract Results
    val_loss, val_acc, val_prec, val_rec = utils.extract_validation_results(history)
    test_loss, test_acc, test_prec, test_rec = model.evaluate(X_test, y_test, batch_size=1024, verbose=0)
    
    # Save To Lists
    n_samples_li.append(grid['number_of_texts'])
    n_hidden_li.append(grid['n_hidden'])
    val_loss_li.append(val_loss)
    val_acc_li.append(val_acc)
    val_prec_li.append(val_prec)
    val_rec_li.append(val_rec)
    test_loss_li.append(test_loss)
    test_acc_li.append(test_acc)
    test_prec_li.append(test_prec)
    test_rec_li.append(test_rec)
    
    # Log Results
    log.info(f"Model {ix} - Prec {test_prec} - Rec {test_rec}")
    log.info(f"Settings - n_samples = {grid['number_of_texts']}, n_hidden = {grid['n_hidden']}")

2019-06-10 13:49:17,862 - INFO - Model 0 - Prec 0.8126036483796238 - Rec 0.8641975308133922
2019-06-10 13:49:17,862 - INFO - Settings - n_samples = 1000, n_hidden = 32


Epoch 00156: early stopping


2019-06-10 13:49:22,563 - INFO - Model 1 - Prec 0.8456790122982223 - Rec 0.885949441452913
2019-06-10 13:49:22,563 - INFO - Settings - n_samples = 5000, n_hidden = 32


Epoch 00084: early stopping


2019-06-10 13:49:29,260 - INFO - Model 2 - Prec 0.8227582590024772 - Rec 0.9223985890110289
2019-06-10 13:49:29,260 - INFO - Settings - n_samples = 10000, n_hidden = 32


...

Epoch 00047: early stopping


2019-06-10 13:50:37,870 - INFO - Model 9 - Prec 0.8382193267731693 - Rec 0.9077013520924337
2019-06-10 13:50:37,871 - INFO - Settings - n_samples = 20000, n_hidden = 64


Epoch 00043: early stopping


2019-06-10 13:50:53,154 - INFO - Model 10 - Prec 0.841738197379735 - Rec 0.9223985890110289
2019-06-10 13:50:53,155 - INFO - Settings - n_samples = 30000, n_hidden = 64


Epoch 00047: early stopping


In [ ]:
import pandas as pd
results_df = pd.DataFrame({
    'n_samples':n_samples_li,
    'n_hidden':n_hidden_li,
    'loss':loss_li,
    'accuracy':acc_li,
    'precision':prec_li,
    'recall':rec_li
}).sort_values('precision',ascending=False)
results_df